In [1]:
!pip3 install polars

In [2]:
import polars as pl
import os

In [3]:
CLEAN_DATA = False

In [4]:
# JOIN DATASETS

if CLEAN_DATA: 
    stations = pl.read_csv("dataset/stations.csv")  
    station_day = pl.read_csv("dataset/station_day.csv")
    df = stations.join(station_day, on="StationId")
    df.write_csv("dataset/full_data.csv", separator=",")

    df = pl.read_csv("dataset/full_data.csv")
    df

In [5]:
# FILTER TO CITIES WITH MORE THAN 3 CITIES

if CLEAN_DATA: 

  df.select(["StationId", "City"]) \
      .group_by("City", "StationId").count()

  cities_wo_3sts = df.select(["StationId", "City"]).unique().group_by("City") \
    .count().filter(pl.col("count") > 3).sort("count")["City"].to_list()

  print(cities_wo_3sts)

  df = df.filter(pl.col("City").is_in(cities_wo_3sts)).select(["StationId", "City", "Date", "Status", "PM2.5", "PM10", "NOx", "CO", "SO2", "O3"])

  df.select(pl.all().is_null().sum()).drop(["StationId", "City", "Date", "Status"])

In [6]:
# FILTERED TO STATIONS WITH MORE THAN 500 ROWS + DROP ROWS WITH NULL VALUES + PICK CITY (DELHI)

if CLEAN_DATA: 
    temp_df = df.group_by("City", "StationId").count().sort("count").filter(pl.col("count") > 500)

    df = df.join(temp_df, on=["StationId", "City"])
    df

    df.drop_nulls().group_by("StationId", "City").count().sort("count")

    df = df.drop_nulls().filter(pl.col("City").is_in(["Delhi"]))
    print(df.shape)
    print("Unique stations in Delhi: ", len(df.select("StationId").unique()["StationId"].to_list()))

    df = df.drop("count")

    df.write_csv("dataset/cleaned_data.csv", separator=",")

In [7]:
df = pl.read_csv("dataset/cleaned_data.csv")
df

StationId,City,Date,Status,PM2.5,PM10,NOx,CO,SO2,O3
str,str,str,str,f64,f64,f64,f64,f64,f64
"""DL001""","""Delhi""","""2018-11-11""","""Active""",232.36,415.28,43.87,1.49,23.48,46.28
"""DL001""","""Delhi""","""2018-11-12""","""Active""",238.43,348.91,44.63,1.53,14.38,45.31
"""DL001""","""Delhi""","""2018-11-13""","""Active""",284.83,427.4,76.15,1.43,21.33,30.4
"""DL001""","""Delhi""","""2018-11-14""","""Active""",150.41,214.16,56.3,0.99,12.74,41.87
"""DL001""","""Delhi""","""2018-11-15""","""Active""",106.11,154.36,47.21,0.7,12.46,27.93
…,…,…,…,…,…,…,…,…,…
"""DL038""","""Delhi""","""2020-06-27""","""Active""",41.85,156.2,30.21,1.55,24.48,12.19
"""DL038""","""Delhi""","""2020-06-28""","""Active""",48.42,284.41,40.5,1.44,24.31,12.58
"""DL038""","""Delhi""","""2020-06-29""","""Active""",42.64,172.11,31.08,1.29,19.87,17.99
